In [ ]:
!pip install -q huggingface_hub

In [ ]:
!pip install smolagents
!pip install ddgs

In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel
from smolagents import tool
from huggingface_hub import list_models



@tool
def my_custom_tool(task: str) -> str:
    """
    Returns the most downloaded model of a given task from Hugging Face Hub.
    Args:
        task: The task name (e.g., "text-generation")
    """
    most_downloaded = next(list_models(filter=task, sort="downloads", direction=-1))
    return most_downloaded.id


model = InferenceClientModel(model_id="Qwen/Qwen3-4B-Thinking-2507")
agent=CodeAgent(tools=[my_custom_tool],model=model)

# Create the agent with the search tool
# agent = CodeAgent(
#     tools=[DuckDuckGoSearchTool()],
#     model=model,
# )

# # Run the agent with your query
# result = agent.run("Who was 1st president of India?")
# print(result)
result = agent.run("Get the most downloaded model for text-generation")
print(result)


In [ ]:
import os
from google.colab import userdata
os.environ['HF_TOKEN'] =userdata.get('HF_TOKEN')

In [ ]:


# from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel
# model = InferenceClientModel(model_id="Qwen/Qwen3-4B-Thinking-2507")
# agent = CodeAgent(
#     model=model,
#     tools=[DuckDuckGoSearchTool()],
# )

# agent.run("How many seconds does clock take to complete one minute?")


In [ ]:
pip install google-generativeai


In [ ]:

import google.generativeai as genai


genai.configure(api_key=api_key)

model = genai.GenerativeModel("gemini-1.5-flash")

response = model.generate_content("Write a short poem about AI and coffee")
print(response.text)


In [ ]:
# agent.run('What is the cube root of 64?')

In [ ]:
pip install litellm


In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel

# Pass your Gemini API key (set it as env variable for safety!)
import os
os.environ["GEMINI_API_KEY"] = "your_api_key_here"

# LiteLLM uses gemini models with the provider prefix "gemini"
model = LiteLLMModel(model_id="gemini/gemini-1.5-flash", api_key=api_key)

agent = CodeAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model
)

result = agent.run("Search for the best music recommendations for a party at a mansion.")
print(result)


Using ToolAgent instead of CodeAgent

In [ ]:
from smolagents import ToolCallingAgent, DuckDuckGoSearchTool, LiteLLMModel
from typing import Optional
model=LiteLLMModel(model_id="gemini/gemini-1.5-flash", api_key=api_key)
agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool()],
    model=model
)
@tool
def get_weather(location:str,celsius:Optional[bool]=False)->str:
    """
    Get weather of the location at present and upcoming days.
    Args:
        location : the location
        celsius : whether to use celsius for temperature
    """
    return f"the weather in {location} is sunny with low temperatures. \n"

# agent=ToolCallingAgent(tools=[get_weather],model=model)
result = agent.run("What's the weather in New York ?")
print(result)



In [ ]:
!pip install gradio


Gradio use to generate image

In [ ]:
from smolagents import ToolCallingAgent,load_tool, DuckDuckGoSearchTool, LiteLLMModel,GradioUI
image_generation_tool=load_tool("m-ric/text-to-image",trust_remote_code=True)
model=LiteLLMModel(model_id="gemini/gemini-1.5-flash", api_key=api_key)
agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(),image_generation_tool],
    model=model
)
GradioUI(agent).launch()

Using Jina AI for web scrapping

In [ ]:
pip install jinaai


In [ ]:
import requests
from smolagents import Tool, ToolCallingAgent, LiteLLMModel


class JinaScrapeTool(Tool):
    name = "scrape_page_with_jina_ai"
    description = "Scrapes a webpage and extracts useful text using Jina AI"

    inputs = {"url": {"type": "string", "description": "The URL to scrape"}}
    output_type = "string"

    def forward(self, url: str) -> str:
        headers = {"Authorization": f"Bearer {"API_KEY"}"}
        resp = requests.get(f"https://r.jina.ai/{url}", headers=headers)
        return resp.text

model = LiteLLMModel(model_id="gemini/gemini-1.5-flash", api_key=api_key)


agent = ToolCallingAgent(
    tools=[JinaScrapeTool()],
    model=model
)

# Now you can run the agent like
result = agent.run("Scrape the text from https://www.digitalocean.com/blog")
print(result)
